In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import datetime
import time
import seaborn as sns; sns.set()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import scipy as sp
import scipy.optimize as optimize
from lmfit import minimize, Parameters, Parameter, report_fit
from scipy.integrate import odeint
from scipy import stats

#import import_ipynb
from utils_common import *

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [40]:
_Figure_PATH_O_ = './figures/'
_Data_PATH_O_ = './data/'
_Data_PATH_XX_ = './data/parameters_xx/'
# m stands for migration
_Data_PATH_MXXR_ = './data/parameters_mxxr/' 
_Data_PATH_MXXRF_ = './data/parameters_mxxr_fast/'

_City_PATH_ = './data/data_DXY_city_all.csv'
_Province_PATH_ = './data/data_DXY_province_all.csv'
_Province_Domestic_PATH_ = './data/data_DXY_province_all_domestic.csv'

In [41]:
# Calcuate the standard errors of parameters (from script: utils_parameters)
# reference: Dueling biological and social contagions, Feng Fu
# a linear approximation through estimation of the Jacobian matrix: 
# cov = sigma^2(F'F)^(-1)
# the unbiased estimation of sigma^2 obtained from the residuals of the parameter estimation:
# sigma = error/sqrt(number of observations - number of free parameters)

def SDE_xx(df_parameters_list, porvince_index, T = 21, TT = 6, TI = 3, ti = 7, figure = False):
    
    df_parameters = df_parameters_list[porvince_index].copy() # the parameters
    df_estimation = df_estimation_list[porvince_index].copy() # the estimation
    
    timespan = df_estimation.shape[0] # national timespan
    timespan_local = timespan - df_estimation['real'].isnull().sum() # provincial timespan
    
    e = np.linalg.norm(df_estimation.tail(timespan_local).residual) # Euclidean Norm of the residual
    k = df_parameters.shape[0] - 2 # number of free parameters (exclude N and S)
    sigma_e = e/np.sqrt(timespan_local - k) # sigma
    
    ps_old = dict(zip(df_parameters.parameter, df_parameters.value))
    para_list = df_parameters.parameter.tolist() 
    para_list.remove('N')
    para_list.remove('S_0')
    
    list_front = np.repeat(range(int(np.ceil(T/TI))), TI).tolist()
    list_end = np.repeat(range(int(np.ceil((timespan - T)/ti))), ti).tolist()
    list_end = [item + max(list_front) + 1 for item in list_end]
    index_list = list_front + list_end
    index_list = index_list[:timespan] # index list for beta
    list_front_gs = [0]*TT + [1]*TT
    list_end_gs = [2]*(timespan - TT - TT)
    index_list_gs = list_front_gs + list_end_gs # index list for gamma and sigma
    
    ts = range(0, timespan)
    nbeta = int(T/TI + np.ceil((timespan - T)/ti)) # number of beta's
    
    # Least-square fitting
    def SEIR_equations(states, t, ps):
        """Receptor synthesis-internalization model."""
        # integrate.ode may generate t values outside the data range
        beta_index = index_list[min(round(t), timespan - 1)]
        gs_index = index_list_gs[min(round(t), timespan - 1)]
        beta = ps['beta_' + str(beta_index)]
        gamma = ps['gamma_' + str(gs_index)]
        sigma = ps['sigma_' + str(gs_index)]
        N = ps['N']
        S, E, I, R = states
        return [-beta*S*I/N, beta*S*I/N - sigma*E, sigma*E - gamma*I, gamma*I]
    
    def SEIR(states_0, ts, ps):
        """
        Solution to the ODE x'(t) = f(t,x,k) with initial condition x(0) = x0
        """
        states = odeint(SEIR_equations, states_0, ts, args=(ps,))
        return states
    
    # Calculate the estimation
    def estimation(ps, ts):
        states_0 = ps['N'] - ps['E_0'] - ps['I_0'] - ps['R_0'], ps['E_0'], ps['I_0'], ps['R_0']
        model = SEIR(states_0, ts, ps)
        est = model[:,3]
        return est
    
    J = np.zeros((timespan_local, k)) # Jacobian matrix
    
    for j in range(0, k):
        para = para_list[j]
        para_value_old = ps_old[para]
        if para_value_old >= 1e-10:
            para_value_change = 1e-2*para_value_old # difference = one percent
        else: # in case the value of a parameter is zero
            para_value_change = 1e-12 
        ps_new = dict(zip(df_parameters.parameter, df_parameters.value))
        ps_new[para] = para_value_old + para_value_change
        final = estimation(ps_new, ts)
        J[:,j] = (df_estimation.tail(timespan_local).estimation - final[-timespan_local:])/para_value_change
        
        palette = plt.get_cmap('magma')
        if figure:
            fig = plt.figure(figsize = (10,5))
            plt.plot(ts, df_estimation.estimation, '-', linewidth = 2, color = palette(0.6))
            plt.plot(ts, final, '--', linewidth = 2, color = palette(0.8))
            plt.title(para)
            plt.xlabel('Date')
            plt.ylabel('Number of people')
        
    H = np.transpose(J).dot(J)
    H_inverse = np.linalg.inv(H)
    df_parameters["sde"] = np.nan
    df_parameters["lower_bound"] = np.nan
    df_parameters["upper_bound"] = np.nan
    
    for j in range(0, k):
        para = para_list[j]
        df_parameters.loc[df_parameters.parameter == para, 'sde'] = sigma_e*np.sqrt(H_inverse[j][j]/ps_old['N'])
    # calculate the lower bound and upper bound of every parameter (99.9% confidence interval)
    # lower bound is non-negative
    # !!! ignore nan !!!
    df_parameters["lower_bound"] = df_parameters.apply(lambda row: np.nanmax([row.value - row.sde*stats.t.ppf(1-0.0005, timespan_local - k), 0]), axis = 1)
    df_parameters["upper_bound"] = df_parameters.apply(lambda row: row.value + row.sde*stats.t.ppf(1-0.0005, timespan_local - k), axis = 1)
    ###### the value of beta cannot exceed 1
    for j in range(max(index_list) + 1):
        df_parameters.loc[df_parameters.parameter == 'beta_' + str(int(j)), 'lower_bound'] = np.nanmax([df_parameters.loc[df_parameters.parameter == 'beta_' + str(int(j)), 'lower_bound'].tolist()[0], 0.01])
        df_parameters.loc[df_parameters.parameter == 'beta_' + str(int(j)), 'upper_bound'] = np.nanmin([df_parameters.loc[df_parameters.parameter == 'beta_' + str(int(j)), 'upper_bound'].tolist()[0], 1])
    ###### the value of gamma and sigma cannot be less than 0.05
    ###### the value of gamma and sigma cannot exceed 0.5
    for j in range(max(index_list_gs) + 1):
        df_parameters.loc[df_parameters.parameter == 'gamma_' + str(int(j)), 'lower_bound'] = np.nanmax([df_parameters.loc[df_parameters.parameter == 'gamma_' + str(int(j)), 'lower_bound'].tolist()[0], 0.05])
        df_parameters.loc[df_parameters.parameter == 'sigma_' + str(int(j)), 'lower_bound'] = np.nanmax([df_parameters.loc[df_parameters.parameter == 'sigma_' + str(int(j)), 'lower_bound'].tolist()[0], 0.05])
        
        df_parameters.loc[df_parameters.parameter == 'gamma_' + str(int(j)), 'upper_bound'] = np.nanmin([df_parameters.loc[df_parameters.parameter == 'gamma_' + str(int(j)), 'upper_bound'].tolist()[0], 0.5])
        df_parameters.loc[df_parameters.parameter == 'sigma_' + str(int(j)), 'upper_bound'] = np.nanmin([df_parameters.loc[df_parameters.parameter == 'sigma_' + str(int(j)), 'upper_bound'].tolist()[0], 0.5])
    return df_parameters

In [42]:
# SEIR simulation WITH MIGRATION given prior estimation of parameters (from script: utils_parameters)
# same timeline for all porvinces
# beta, gamma and sigma are time-dependent
# beta: first changes every TI = 3 days for T = 21 days, then changes every ti days
# gamma and sigma: first changes every TT = 6 days for 2*TT = 12 days, then stays the same
###### recursive method:
# one complete cycle: from the first province to the last province
# repetition: n cycle
###### algorithm:
# ’leastsq’: Levenberg-Marquardt (default)
def simulation_xxrecursive(df, update_date_tr, mindex_list, repetition = 1,
                 T = 21, TT = 6, TI = 3, ti = 7, mmethod = 'leastsq', kws = False):
    
    start_time = time.time() 
    m = len(names_province)
    df = df[df.update_date <= update_date_tr] 
    timespan = (max(df.update_date) - min(df.update_date)).days + 1
    ################## for beta, generate an index list
    list_front = np.repeat(range(int(np.ceil(T/TI))), TI).tolist()
    list_end = np.repeat(range(int(np.ceil((timespan - T)/ti))), ti).tolist()
    list_end = [item + max(list_front) + 1 for item in list_end]
    index_list = list_front + list_end
    index_list = index_list[:timespan] 
    ################## 
    ################## for sigma and gamma, generate an index list
    list_front_gs = [0]*TT + [1]*TT
    list_end_gs = [2]*(timespan - TT - TT)
    index_list_gs = list_front_gs + list_end_gs
    ################## 
    df_single_list = [] # list of dataframes 
    ts_local_list = [] # list of provincial timespans
    data = np.zeros((timespan, m)) # cumulative numbers of confirmed cases
    
    for i, name in enumerate(names_province):
        df_single = df[df.province_name_en == name]
        timespan_local = (max(df_single.update_date) - min(df_single.update_date)).days + 1
        df_single_list.append(df_single)
        ts_local_list.append(timespan_local)
        data_single = np.array(df_single.cum_confirmed)
        data_single = np.concatenate(([None]*(timespan - timespan_local), data_single), axis=0)
        data[:,i] = data_single
    
    # Least-square fitting
    # SEIR model with migration
    def SEIR_equations(states, t, ps):
        """Receptor synthesis-internalization model."""
        #S = [0]*m; E = [0]*m # given that there are m provinces, at every time point, 
        #I = [0]*m; R = [0]*m # the variable is a vector of length m
        states = states.reshape((m, 4)) # S_i, E_i, I_i, R_i, S_j, E_j, I_j, R_j
        S, E, I, R = states[:,0], states[:,1], states[:,2], states[:,3]
        N = S + E + I + R
        RHS = [] # right hand side
        theta = ps['theta'].value
        # integrate.ode may generate t values outside the data range
        mindex = mindex_list[min(round(t), timespan - 1)] # migration matrix: m_ij from i to j
        for i, name in enumerate(names_province):
            # integrate.ode may generate t values outside the data range
            beta_index = index_list[min(round(t), timespan - 1)]
            gs_index = index_list_gs[min(round(t), timespan - 1)]
            beta = ps['beta_' + str(i) + '_' + str(beta_index)].value
            gamma = ps['gamma_' + str(i) + '_' + str(gs_index)].value
            sigma = ps['sigma_' + str(i) + '_' + str(gs_index)].value
            
            dS = -beta*S[i]*I[i]/N[i] - theta/N[i]*np.sum(mindex[i, :])*S[i] + theta*np.sum(np.multiply(np.divide(mindex[:,i], N), S))
            dE = beta*S[i]*I[i]/N[i] - sigma*E[i] - theta/N[i]*np.sum(mindex[i, :])*E[i] + theta*np.sum(np.multiply(np.divide(mindex[:,i], N), E))
            dI = sigma*E[i] - gamma*I[i]
            dR = gamma*I[i]
            RHS += [dS, dE, dI, dR]
        return RHS
    
    def SEIR(states_0, ts, ps):
        """
        Solution to the ODE x'(t) = f(t,x,k) with initial condition x(0) = x0
        """
        states = odeint(SEIR_equations, states_0, ts, args=(ps,))
        return states
    
    # Calculate the residual
    def residual(ps, ts, data):
        states_0 = []
        for i, name in enumerate(names_province):
            states_single_0 = ps['S_' + str(i) + '_0'].value, ps['E_' + str(i) + '_0'].value, ps['I_' + str(i) + '_0'].value, ps['R_' + str(i) + '_0'].value
            states_0 += states_single_0
        model = SEIR(states_0, ts, ps)
        res = np.empty(0)
        for i, name in enumerate(names_province):
            timespan_local = ts_local_list[i]
            est_single = model[:,(3 + i*4)]
            est_single = est_single[-timespan_local:]
            data_single = data[:,i]
            data_single = data_single[-timespan_local:]
            res_single = (est_single - data_single).ravel()
            res_single = res_single/data_single[-1] # normalization
            res = np.concatenate((res, res_single), axis=0)
        return res
    
    # Calculate the estimation
    def estimation(ps, ts):
        states_0 = []
        for i, name in enumerate(names_province):
            states_single_0 = ps['S_' + str(i) + '_0'].value, ps['E_' + str(i) + '_0'].value, ps['I_' + str(i) + '_0'].value, ps['R_' + str(i) + '_0'].value
            states_0 += states_single_0
        model = SEIR(states_0, ts, ps)
        est = np.zeros((timespan, m))
        for i, name in enumerate(names_province):
            est_single = model[:,(3 + i*4)]
            est[:, i] = est_single
        return est
    
    # recursively performing optimizations 
    df_parameters_list_r = [df_parameters_list[i].copy() for i in range(0, len(names_province))]
    # repetition
    for flag in range(0, repetition):
        # one complete cycle: from the first province to the last province
        # the k th step: only the parameters of province k is changable, those of the other provinces are fixed
        for k, name in enumerate(names_province): 
            # set parameters (incluing their bounds)
            ps_bound = Parameters()
            for i, name in enumerate(names_province):
                prior = df_parameters_list_r[i]
                ps_bound.add(name = 'N_' + str(i) + '_0', value = prior[prior.parameter == 'N'].value.tolist()[0], vary = False)
                if i != k:
                    ps_bound.add(name = 'E_' + str(i) + '_0', value = prior[prior.parameter == 'E_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'I_' + str(i) + '_0', value = prior[prior.parameter == 'I_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'R_' + str(i) + '_0', value = prior[prior.parameter == 'R_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'S_' + str(i) + '_0', expr = 'N_' + str(i) + '_0' + '-' 'E_' + str(i) + '_0' + '-' 'I_' + str(i) + '_0' + '-' 'R_' + str(i) + '_0')
                    for j in range(max(index_list) + 1):
                        ps_bound.add(name = 'beta_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'beta_' + str(int(j))].value.tolist()[0], vary = False)
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'gamma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'gamma_' + str(int(j))].value.tolist()[0], vary = False)
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'sigma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'sigma_' + str(int(j))].value.tolist()[0], vary = False)
                else:
                    ps_bound.add(name = 'E_' + str(i) + '_0', value = prior[prior.parameter == 'E_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([500, prior[prior.parameter == 'E_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'I_' + str(i) + '_0', value = prior[prior.parameter == 'I_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([200, prior[prior.parameter == 'I_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'R_' + str(i) + '_0', value = prior[prior.parameter == 'R_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([100, prior[prior.parameter == 'R_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'S_' + str(i) + '_0', expr = 'N_' + str(i) + '_0' + '-' 'E_' + str(i) + '_0' + '-' 'I_' + str(i) + '_0' + '-' 'R_' + str(i) + '_0')
                    for j in range(max(index_list) + 1):
                        ps_bound.add(name = 'beta_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'beta_' + str(int(j))].value.tolist()[0], 
                                     min = 0.01, # prior[prior.parameter == 'beta_' + str(int(j))].lower_bound.tolist()[0]
                                     max = prior[prior.parameter == 'beta_' + str(int(j))].upper_bound.tolist()[0])
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'gamma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'gamma_' + str(int(j))].value.tolist()[0], 
                                     min = 0.05, # prior[prior.parameter == 'gamma_' + str(int(j))].lower_bound.tolist()[0] 
                                     max = prior[prior.parameter == 'gamma_' + str(int(j))].upper_bound.tolist()[0])
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'sigma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'sigma_' + str(int(j))].value.tolist()[0], 
                                     min = 0.05, # prior[prior.parameter == 'sigma_' + str(int(j))].lower_bound.tolist()[0]
                                     max = prior[prior.parameter == 'sigma_' + str(int(j))].upper_bound.tolist()[0])

            # the unit of migration index is 1e5 (see utils_migration for explanation)
            ps_bound.add(name = 'theta', value = 1e5, vary = False)
            ts = range(0, timespan)
            # fit the model and find estimated values
            if kws == False:
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod)
            elif mmethod == 'emcee':
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod, steps = kws.get('steps'))
            elif mmethod == 'dual_annealing':
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod, maxiter = kws.get('maxiter'))
            # the dataframe above is a long dataframe consisting of parameters for all provinces
            df_parameters = pd.DataFrame(dict(result.params.valuesdict()).items(), columns=['parameter', 'value'])
            # only update the parameters for the k th province
            start_index = df_parameters.index[df_parameters['parameter'] == 'N_' + str(k) + '_0'].tolist()[0]
            end_index = df_parameters.index[df_parameters['parameter'] == 'sigma_' + str(k) + '_' + str(max(index_list_gs))].tolist()[0]
            update = df_parameters[start_index:(end_index + 1)]
            update = update.reset_index(drop = True) ###### !!!!!! ######
            df_parameters_list_r[k]['value'] = update['value']
            print("province %d of round %d" % (k, flag))
            print("--- %s seconds ---" % (time.time() - start_time))
            
    final = estimation(result.params, ts)
    _Data_PATH_MXXR_ = './data/parameters_mxxr_' + str(repetition) + '/' 
    
    df_mestimation_list = [] # m for migration
    for i, name in enumerate(names_province):
        df_mestimation = df_estimation_list[i].copy()
        df_mestimation['mestimation'] = final[:, i]
        df_mestimation_list.append(df_mestimation)
        df_mestimation.to_csv(_Data_PATH_MXXR_ + name + '_mestimation.csv', index = False)
        df_parameters_list_r[i].to_csv(_Data_PATH_MXXR_ + name + '_mparameters.csv', index = False)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    df_parameters.to_csv(_Data_PATH_MXXR_ + 'mparameters.csv', index = False)

    return result, df_mestimation_list, df_parameters_list_r


In [43]:
# SEIR simulation WITH MIGRATION given prior estimation of parameters (from script: utils_parameters)
# same timeline for all porvinces
# beta, gamma and sigma are time-dependent
# beta: first changes every TI = 3 days for T = 21 days, then changes every ti days
# gamma and sigma: first changes every TT = 6 days for 2*TT = 12 days, then stays the same
###### recursive method:
# one complete cycle: from the first province to the last province
# repetition: n cycle
###### algorithm:
# ’leastsq’: Levenberg-Marquardt (default)
###### fast:
# only the first few beta's, gamma's and sigma's are changable
def simulation_xxrecursive_fast(df, update_date_tr, mindex_list, repetition = 1,
                 T = 21, TT = 6, TI = 3, ti = 7, mmethod = 'leastsq', kws = False):
    
    start_time = time.time() 
    m = len(names_province)
    df = df[df.update_date <= update_date_tr] 
    timespan = (max(df.update_date) - min(df.update_date)).days + 1
    ################## for beta, generate an index list
    list_front = np.repeat(range(int(np.ceil(T/TI))), TI).tolist()
    list_end = np.repeat(range(int(np.ceil((timespan - T)/ti))), ti).tolist()
    list_end = [item + max(list_front) + 1 for item in list_end]
    index_list = list_front + list_end
    index_list = index_list[:timespan] 
    ################## 
    ################## for sigma and gamma, generate an index list
    list_front_gs = [0]*TT + [1]*TT
    list_end_gs = [2]*(timespan - TT - TT)
    index_list_gs = list_front_gs + list_end_gs
    ################## 
    df_single_list = [] # list of dataframes 
    ts_local_list = [] # list of provincial timespans
    data = np.zeros((timespan, m)) # cumulative numbers of confirmed cases
    
    for i, name in enumerate(names_province):
        df_single = df[df.province_name_en == name]
        timespan_local = (max(df_single.update_date) - min(df_single.update_date)).days + 1
        df_single_list.append(df_single)
        ts_local_list.append(timespan_local)
        data_single = np.array(df_single.cum_confirmed)
        data_single = np.concatenate(([None]*(timespan - timespan_local), data_single), axis=0)
        data[:,i] = data_single
    
    # Least-square fitting
    # SEIR model with migration
    def SEIR_equations(states, t, ps):
        """Receptor synthesis-internalization model."""
        #S = [0]*m; E = [0]*m # given that there are m provinces, at every time point, 
        #I = [0]*m; R = [0]*m # the variable is a vector of length m
        states = states.reshape((m, 4)) # S_i, E_i, I_i, R_i, S_j, E_j, I_j, R_j
        S, E, I, R = states[:,0], states[:,1], states[:,2], states[:,3]
        N = S + E + I + R
        RHS = [] # right hand side
        theta = ps['theta'].value
        # integrate.ode may generate t values outside the data range
        mindex = mindex_list[min(round(t), timespan - 1)] # migration matrix: m_ij from i to j
        for i, name in enumerate(names_province):
            # integrate.ode may generate t values outside the data range
            beta_index = index_list[min(round(t), timespan - 1)]
            gs_index = index_list_gs[min(round(t), timespan - 1)]
            beta = ps['beta_' + str(i) + '_' + str(beta_index)].value
            gamma = ps['gamma_' + str(i) + '_' + str(gs_index)].value
            sigma = ps['sigma_' + str(i) + '_' + str(gs_index)].value
            
            dS = -beta*S[i]*I[i]/N[i] - theta/N[i]*np.sum(mindex[i, :])*S[i] + theta*np.sum(np.multiply(np.divide(mindex[:,i], N), S))
            dE = beta*S[i]*I[i]/N[i] - sigma*E[i] - theta/N[i]*np.sum(mindex[i, :])*E[i] + theta*np.sum(np.multiply(np.divide(mindex[:,i], N), E))
            dI = sigma*E[i] - gamma*I[i]
            dR = gamma*I[i]
            RHS += [dS, dE, dI, dR]
        return RHS
    
    def SEIR(states_0, ts, ps):
        """
        Solution to the ODE x'(t) = f(t,x,k) with initial condition x(0) = x0
        """
        states = odeint(SEIR_equations, states_0, ts, args=(ps,))
        return states
    
    # Calculate the residual
    def residual(ps, ts, data):
        states_0 = []
        for i, name in enumerate(names_province):
            states_single_0 = ps['S_' + str(i) + '_0'].value, ps['E_' + str(i) + '_0'].value, ps['I_' + str(i) + '_0'].value, ps['R_' + str(i) + '_0'].value
            states_0 += states_single_0
        model = SEIR(states_0, ts, ps)
        res = np.empty(0)
        for i, name in enumerate(names_province):
            timespan_local = ts_local_list[i]
            est_single = model[:,(3 + i*4)]
            est_single = est_single[-timespan_local:]
            data_single = data[:,i]
            data_single = data_single[-timespan_local:]
            res_single = (est_single - data_single).ravel()
            res_single = res_single/data_single[-1] # normalization
            res = np.concatenate((res, res_single), axis=0)
        return res
    
    # Calculate the estimation
    def estimation(ps, ts):
        states_0 = []
        for i, name in enumerate(names_province):
            states_single_0 = ps['S_' + str(i) + '_0'].value, ps['E_' + str(i) + '_0'].value, ps['I_' + str(i) + '_0'].value, ps['R_' + str(i) + '_0'].value
            states_0 += states_single_0
        model = SEIR(states_0, ts, ps)
        est = np.zeros((timespan, m))
        for i, name in enumerate(names_province):
            est_single = model[:,(3 + i*4)]
            est[:, i] = est_single
        return est
    
    # recursively performing optimizations 
    df_parameters_list_r = [df_parameters_list[i].copy() for i in range(0, len(names_province))]
    # repetition
    for flag in range(0, repetition):
        # one complete cycle: from the first province to the last province
        # the k th step: only the parameters of province k is changable, those of the other provinces are fixed
        for k, name in enumerate(names_province): 
            # set parameters (incluing their bounds)
            ps_bound = Parameters()
            for i, name in enumerate(names_province):
                prior = df_parameters_list_r[i]
                ps_bound.add(name = 'N_' + str(i) + '_0', value = prior[prior.parameter == 'N'].value.tolist()[0], vary = False)
                if i != k:
                    ps_bound.add(name = 'E_' + str(i) + '_0', value = prior[prior.parameter == 'E_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'I_' + str(i) + '_0', value = prior[prior.parameter == 'I_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'R_' + str(i) + '_0', value = prior[prior.parameter == 'R_0'].value.tolist()[0], vary = False)
                    ps_bound.add(name = 'S_' + str(i) + '_0', expr = 'N_' + str(i) + '_0' + '-' 'E_' + str(i) + '_0' + '-' 'I_' + str(i) + '_0' + '-' 'R_' + str(i) + '_0')
                    for j in range(max(index_list) + 1):
                        ps_bound.add(name = 'beta_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'beta_' + str(int(j))].value.tolist()[0], vary = False)
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'gamma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'gamma_' + str(int(j))].value.tolist()[0], vary = False)
                    for j in range(max(index_list_gs) + 1):
                        ps_bound.add(name = 'sigma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'sigma_' + str(int(j))].value.tolist()[0], vary = False)
                else:
                    ps_bound.add(name = 'E_' + str(i) + '_0', value = prior[prior.parameter == 'E_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([500, prior[prior.parameter == 'E_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'I_' + str(i) + '_0', value = prior[prior.parameter == 'I_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([200, prior[prior.parameter == 'I_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'R_' + str(i) + '_0', value = prior[prior.parameter == 'R_0'].value.tolist()[0], 
                                 min = 0, max = np.nanmax([100, prior[prior.parameter == 'R_0'].upper_bound.tolist()[0]]))
                    ps_bound.add(name = 'S_' + str(i) + '_0', expr = 'N_' + str(i) + '_0' + '-' 'E_' + str(i) + '_0' + '-' 'I_' + str(i) + '_0' + '-' 'R_' + str(i) + '_0')
                    # only the first four beta's are changable
                    for j in range(max(index_list) + 1):
                        if j >=4:
                            ps_bound.add(name = 'beta_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'beta_' + str(int(j))].value.tolist()[0], vary = False)
                        else:
                            ps_bound.add(name = 'beta_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'beta_' + str(int(j))].value.tolist()[0], 
                                     min = 0.01, # prior[prior.parameter == 'beta_' + str(int(j))].lower_bound.tolist()[0]
                                     max = prior[prior.parameter == 'beta_' + str(int(j))].upper_bound.tolist()[0])
                    # only the first two gamma's are changable
                    for j in range(max(index_list_gs) + 1):
                        if j >=2:
                            ps_bound.add(name = 'gamma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'gamma_' + str(int(j))].value.tolist()[0], vary = False)
                        else:
                            ps_bound.add(name = 'gamma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'gamma_' + str(int(j))].value.tolist()[0], 
                                     min = 0.05, # prior[prior.parameter == 'gamma_' + str(int(j))].lower_bound.tolist()[0] 
                                     max = prior[prior.parameter == 'gamma_' + str(int(j))].upper_bound.tolist()[0])
                    # only the first two sigma's are changable
                    for j in range(max(index_list_gs) + 1):
                        if j>=2:
                            ps_bound.add(name = 'sigma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'sigma_' + str(int(j))].value.tolist()[0], vary = False)
                        else:
                            ps_bound.add(name = 'sigma_' + str(i) + '_' + str(int(j)), value = prior[prior.parameter == 'sigma_' + str(int(j))].value.tolist()[0], 
                                     min = 0.05, # prior[prior.parameter == 'sigma_' + str(int(j))].lower_bound.tolist()[0]
                                     max = prior[prior.parameter == 'sigma_' + str(int(j))].upper_bound.tolist()[0])

            
            # the unit of migration index is 1e5 (see utils_migration for explanation)
            ps_bound.add(name = 'theta', value = 1e5, vary = False)
            ts = range(0, timespan)
            # fit the model and find estimated values
            if kws == False:
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod)
            elif mmethod == 'emcee':
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod, steps = kws.get('steps'))
            elif mmethod == 'dual_annealing':
                result = minimize(residual, ps_bound, args = (ts, data), method = mmethod, maxiter = kws.get('maxiter'))
            # the dataframe above is a long dataframe consisting of parameters for all provinces
            df_parameters = pd.DataFrame(dict(result.params.valuesdict()).items(), columns=['parameter', 'value'])
            # only update the parameters for the k th province
            start_index = df_parameters.index[df_parameters['parameter'] == 'N_' + str(k) + '_0'].tolist()[0]
            end_index = df_parameters.index[df_parameters['parameter'] == 'sigma_' + str(k) + '_' + str(max(index_list_gs))].tolist()[0]
            update = df_parameters[start_index:(end_index + 1)]
            update = update.reset_index(drop = True) ###### !!!!!! ######
            df_parameters_list_r[k]['value'] = update['value']
            print("province %d of round %d" % (k, flag))
            print("--- %s seconds ---" % (time.time() - start_time))   
    
    final = estimation(result.params, ts)
    
    _Data_PATH_MXXRF_ = './data/parameters_mxxr_fast_' + str(repetition) + '/' 
    
    df_mestimation_list = [] # m for migration
    for i, name in enumerate(names_province):
        df_mestimation = df_estimation_list[i].copy()
        df_mestimation['mestimation'] = final[:, i]
        df_mestimation_list.append(df_mestimation)
        df_mestimation.to_csv(_Data_PATH_MXXRF_ + name + '_mestimation.csv', index = False) # F stands for fast
        df_parameters_list_r[i].to_csv(_Data_PATH_MXXRF_ + name + '_mparameters.csv', index = False)
    
    print("--- %s seconds ---" % (time.time() - start_time))

    df_parameters.to_csv(_Data_PATH_MXXRF_ + 'parameters.csv', index = False)

    return result, df_mestimation_list, df_parameters_list_r


In [60]:
def figure_conf_est_all(df, names_province, update_date_tr, fsize = (7, 2), ncol = 3, ms = 3, fs = 12, 
                        title = None, country = 'China', tag = 'without', repetition = 10):
    
    sns.set_style("ticks")
    palette = plt.get_cmap('Reds')
    palette_est = plt.get_cmap('Blues')
    palette_mest = plt.get_cmap('Greens')
    palette = ['#679b9b', '#aacfcf', '#fde2e2', '#ffb6b6']
    
    m = len(names_province)
    
    fig, axes = plt.subplots(int(np.ceil(m/ncol)), ncol, figsize = (2*fsize[0], int(np.ceil(m/ncol))*fsize[1]), sharey = False)
    fig.subplots_adjust(hspace = 0.2, wspace = 0.1)
    if m%ncol != 0:
        for j in range(m, int(np.ceil(m/ncol)*ncol)):
            fig.delaxes(axes.flatten()[j])
    
    df = df[df.update_date <= update_date_tr]
    df_rank = df[df['update_date'] == max(df['update_date'])].copy()
    df_rank = df_rank.sort_values(by = 'province_name_en') # same order as the list names_province
    df_rank = df_rank.reset_index(drop=True)
    df_rank = df_rank.sort_values(by = 'cum_confirmed')
    rank_list = df_rank.index.tolist()
    
    date_initial = min(df.update_date)
    plot_df_list = []
    
    for i, province in enumerate(names_province):
        
        ix = np.unravel_index(i, axes.shape)
        #c = palette(rank_list.index(i)/2/m + 0.3)
        #c_est = palette_est(rank_list.index(i)/2/m + 0.3)
        #c_mest = palette_mest(rank_list.index(i)/2/m + 0.3)
        
        if tag == 'without':
            df_estimation = df_estimation_list[i]
        else:
            df_estimation = df_mestimation_list[i]
        
        plot_df = df[df['province_name_en'] == province].copy()
        ##########################################
        if date_initial - min(plot_df.update_date) != datetime.timedelta(0):
            df_initial = pd.DataFrame([[np.nan] * len(plot_df.columns)], columns = plot_df.columns)
            plot_df = df_initial.append(plot_df, ignore_index = True)
            plot_df.iloc[0, plot_df.columns.get_loc('update_date')] = date_initial
            plot_df['update_date'] =  pd.to_datetime(plot_df['update_date'])
            plot_df.set_index('update_date', inplace=True)
            plot_df = plot_df.resample('D').ffill().reset_index()
            plot_df['update_date'] = plot_df['update_date'].dt.date
        ##########################################
        plot_df = plot_df.reset_index()
        plot_df['SEIR'] = df_estimation['estimation']
        if 'mestimation' in df_estimation.columns:
            plot_df['MSEIR'] = df_estimation['mestimation']
        
        plot_df = plot_df.replace({'cum_confirmed': {0: None}})
        
        plot_df_list.append(plot_df)
        
        #axes[ix].plot(plot_df['update_date'], plot_df['cum_confirmed'],
                #linewidth = 2, marker = 'o', ms = ms, color = c, label = (lambda x: None if x > 0 else 'real data')(i)) # 
        axes[ix].fill_between(plot_df['update_date'], plot_df['cum_confirmed'],
                                color = palette[2], alpha = 0.6, label = (lambda x: None if x > 0 else 'real')(i))
        
        axes[ix].plot(plot_df['update_date'], plot_df['SEIR'],
                linewidth = 2, linestyle = '--', marker = '', ms = ms, color = palette[1], label = (lambda x: None if x > 0 else 'estimation w/o migration')(i))
        #axes[ix].bar(plot_df['update_date'], height = plot_df['SEIR'], color = palette[1], alpha = 0.8, width = 0.4, 
                #label = (lambda x: None if x > 0 else 'estimation w/o migration')(i))
        #axes[ix].fill_between(plot_df['update_date'], plot_df['SEIR'],
                                #color = palette[1], alpha = 0.6, label = (lambda x: None if x > 0 else 'estimation w/o migration')(i))
            
        if 'mestimation' in df_estimation.columns:
            axes[ix].plot(plot_df['update_date'], plot_df['MSEIR'],
                linewidth = 2, marker = '', ms = ms, color = palette[0], label = (lambda x: None if x > 0 else 'estimation w migration')(i)) # 
            #axes[ix].fill_between(plot_df['update_date'], plot_df['MSEIR'],
                                #color = palette[2], alpha = 0.6, label = (lambda x: None if x > 0 else 'estimation w migration')(i))
        
            
        if i >= (np.ceil(m/ncol) - 1)*ncol:
            axes[ix].set_xlabel('Date', fontsize = fs - 2)
            
        if i % ncol == 0:
            axes[ix].set_ylabel('Number of people', fontsize = fs - 2)
            axes[ix].get_yaxis().set_label_coords(-0.15,0.5)
        if i == 0:
            axes[ix].legend(loc = 'upper left', fancybox = True, fontsize = fs - 4)
        axes[ix].set_title(province, fontsize = fs)
        axes[ix].set_xlim(min(df.update_date), max(df.update_date))
        axes[ix].xaxis.set_major_locator(mdates.WeekdayLocator())
        axes[ix].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
        axes[ix].tick_params(axis = 'both', which = 'major', labelsize = fs - 4)
        axes[ix].tick_params(axis = 'both', which = 'minor', labelsize = fs - 4)
        
    fig.align_ylabels(axes[:, 0])
        
    fig.suptitle(title, fontsize = fs + 2, y = 1.01)
    if 'mestimation' in df_estimation.columns:
        fig.savefig(_Figure_PATH_O_ + country + '_conf_mseir_' + str(repetition) + '.png', dpi = 400)
    else:
        fig.savefig(_Figure_PATH_O_ + country + '_conf_seir_' + str(repetition) + '.png', dpi = 400)

In [45]:
data_city, data_province, data_province_domestic = load_DXY_raw()

In [46]:
date_initial = min(data_province_domestic.update_date)
date_tr = datetime.date(int(2020),int(3),int(10))

In [47]:
df_parameters_list, df_estimation_list = load_ind_simulation_raw_xx()
df_parameters_list = [SDE_xx(df_parameters_list, i) for i in range(0, len(names_province))]

In [48]:
data_network_P2P = load_network_raw()
mindex_list_real = matrix_P2P_all(data_network_P2P, date_initial, date_tr)

In [112]:
#annealing_kws = dict(maxiter = 10)

In [ ]:
rep = 1
result, df_mestimation_list, df_mparameters_list = simulation_xxrecursive(df = data_province_domestic, 
                                                                           update_date_tr = date_tr, 
                                                                           mindex_list = mindex_list_real, 
                                                                           repetition = rep,
                                                                           T = 21, TT = 6, TI = 3, ti = 7, 
                                                                           mmethod = 'leastsq', kws = False)

In [ ]:
figure_conf_est_all(data_province_domestic, names_province, date_tr, fsize = (7, 2), ncol = 3, ms = 3, fs = 12, 
                    title = 'Mainland China: domestic infection', country = 'China', 
                    tag = 'with', repetition = rep)